### Using the JSONEncoder Class

Recall...when working with serialization, we could use dump or JSONEncoder

Similarly, we can create a custom JSONDecoder class and specify it with the cls argument

In [ ]:
json.loads(j, cls=CustomeDecoder)

It's just a different way of doing it -> might help making sure we use our custom decoder consistently
- It works differently than JSONEncoder
 - We're going to inherit from the JSONDecoder just like we did with the encoder
 - Then we're going to override the decode function
 - **The difference here is** that the decode function receives the entire JSON string
 - So we have to fully parse and return whatever object we want

#### Code Examples

In [1]:
import json

In [2]:
j = '''
{
    "a": 100,
    "b": [1, 2, 3],
    "c": "python",
    "d": {
        "e": 4,
        "f": 5.5
    }
}
'''

In [3]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        print("decode: ", type(arg), arg)
        return "a simple string object"

In [5]:
result = json.loads(j, cls=CustomDecoder)

decode:  <class 'str'> 
{
    "a": 100,
    "b": [1, 2, 3],
    "c": "python",
    "d": {
        "e": 4,
        "f": 5.5
    }
}



In [6]:
result

'a simple string object'

In [9]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [10]:
j_points= '''
{
    "points": [
        [10, 20],
        [-1, -2],
        [0.5, 0.5]
    ]
}
'''

j_other = '''
{
    "a": 1,
    "b": 2
}
'''

In [11]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        if 'points' in arg:
            obj = json.loads(arg)
            return "parsing object for points"
        else:
            return super().decode(arg)

In [13]:
json.loads(j_points, cls=CustomDecoder)

'parsing object for points'

In [14]:
json.loads(j_other, cls=CustomDecoder)

{'a': 1, 'b': 2}

In [16]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        if 'points' in obj:
            obj['points'] = [Point(x, y)
                             for x, y in obj['points']]
        return obj

In [17]:
json.loads(j_points, cls=CustomDecoder)

{'points': [Point(x=10, y=20), Point(x=-1, y=-2), Point(x=0.5, y=0.5)]}

In [18]:
json.loads(j_other,cls=CustomDecoder)

{'a': 1, 'b': 2}

In [19]:
json.loads(j,cls=CustomDecoder)

{'a': 100, 'b': [1, 2, 3], 'c': 'python', 'd': {'e': 4, 'f': 5.5}}

In [20]:
import re

In [21]:
pattern = r'"_type"\s*:\s*"point"'

r is for raw string

In [22]:
print('word\tword')

word	word


In [23]:
print(r'word\tword')

word\tword


In [25]:
regexp = re.compile(pattern)

In [26]:
print(regexp.search('"a":1'))

None


In [27]:
print(regexp.search('"_type": 1'))

None


In [28]:
print(regexp.search('"_type": "point"'))

<re.Match object; span=(0, 16), match='"_type": "point"'>


In [29]:
print(regexp.search('"_type"       :\t "point"'))

<re.Match object; span=(0, 24), match='"_type"       :\t "point"'>


In [30]:
re.search(pattern, '"_type"       :\t "point"')

<re.Match object; span=(0, 24), match='"_type"       :\t "point"'>

In [31]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if '_type' in obj and obj['_type'] == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
                
        return obj

In [38]:
j = '''
{
    "a": 100,
    "b": 0.5,
    "rectangle": {
        "corners": {
            "b_left": {"_type": "point", "x": -1, "y": -1},
            "b_right": {"_type": "point", "x": 1, "y": -1},
            "t_left": {"_type": "point", "x": -1, "y": 1},
            "t_right": {"_type": "point", "x": 1, "y": 1}
        },
        "rotate": {"_type": "point", "x": 0, "y": 0},
        "interior_pts": [
            {"_type": "point", "x": 0, "y": 0},
            {"_type": "point", "x": 0.5, "y": 0.5}
        ]
    }
}
'''

In [39]:
json.loads(j)

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
   'b_right': {'_type': 'point', 'x': 1, 'y': -1},
   't_left': {'_type': 'point', 'x': -1, 'y': 1},
   't_right': {'_type': 'point', 'x': 1, 'y': 1}},
  'rotate': {'_type': 'point', 'x': 0, 'y': 0},
  'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
   {'_type': 'point', 'x': 0.5, 'y': 0.5}]}}

In [42]:
from pprint import pprint
pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [43]:
from decimal import Decimal

In [44]:
CustomDecoder = json.JSONDecoder(parse_float=Decimal)

In [45]:
d = CustomDecoder.decode(j)

In [47]:
pprint(d)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
                           'b_right': {'_type': 'point', 'x': 1, 'y': -1},
                           't_left': {'_type': 'point', 'x': -1, 'y': 1},
                           't_right': {'_type': 'point', 'x': 1, 'y': 1}},
               'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
                                {'_type': 'point',
                                 'x': Decimal('0.5'),
                                 'y': Decimal('0.5')}],
               'rotate': {'_type': 'point', 'x': 0, 'y': 0}}}


In [48]:
class CustomDecoder(json.JSONDecoder):
    base_decoder = json.JSONDecoder(parse_float=Decimal)
    
    def decode(self, arg):
        obj = self.base_decoder.decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if '_type' in obj and obj['_type'] == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
                
        return obj

In [50]:
pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [51]:
result = json.loads(j, cls=CustomDecoder)

In [52]:
pt = result['rectangle']['interior_pts'][1]

In [54]:
type(pt.x)

decimal.Decimal

In [55]:
class CustomDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(parse_float=Decimal)
    
    def decode(self, arg):
        obj = super().decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj = self.make_pts(obj)
        return obj
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if '_type' in obj and obj['_type'] == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_pts(item)
                
        return obj

In [56]:
result = json.loads(j, cls=CustomDecoder)

In [57]:
pprint(result)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [59]:
pt = result['rectangle']['interior_pts'][1]
type(pt.y)

decimal.Decimal